
Código adaptado a partir de turorial encontrado em https://medium.com/@PedroLFerreira/how-to-simulate-wildfires-with-python-6562e2eed266

    Nesse ambiente é criada a lógica para a propagação do fogo partindo do cenário inicial 
    construído a partir da Grid possibilitada pelo pygame.
    
    Os valores das células, atribuidos com o desenho em diferentes cores que possuem valores 
    relacionados, se encontram na matriz que será manipulada.
    
    O tempo de simulação será utilizado para comparar diferentes versões da matriz no tempo 
    e definir o curso da simulação, além de possibilitar a construção de um gif com o uso da 
    biblioteca imageio.

    Após detectado o fogo, é averiguado nas células vizinhas (acima, abaixo, à direita e à 
    esquerda) seu valor correspondente para definição de como será a propagação. 
    
    Caso seja uma vegetação rasteira, o fogo se propaga imediatamente. Caso se trate de uma 
    floresta, o fogo demora mais um pouco para se propagar. Caso se trate de lagos, faixas de 
    terra ou faixas queimadas, o fogo não se propaga.

    A faixa dourada pode ser utilizada para definição de uma área de conservação em que não se 
    deseja que o fogo alcance de forma alguma. Portanto, a simulação é encerrada caso o fogo 
    alcance algum limite dessa faixa. O objetivo é estudar formas de se impedir que isso 
    aconteça, e, caso ocorra, recomeçar os estudos para buscar alternativas.


    


In [5]:
import numpy as np
import imageio
import grid

In [6]:
found_protected = False

u = True if grid.win_dir == 'up' else False # up
d = True if grid.win_dir == 'down' else False # down
l = True if grid.win_dir == 'left' else False # right
r = True if grid.win_dir == 'right' else False # left

if grid.win_dir == 'reset':
    u,d,l,r == True # sem vento: todas as direções iguais

In [ ]:
for t in range(1,grid.total_time):
    if not(found_protected):
        grid.states[t] = grid.states[t-1].copy()

        for i in range(1,grid.terrain_size[0]-1):
            for j in range(1,grid.terrain_size[1]-1):
                if grid.states[t-1,i,j] == 100: # it's on fire
                    grid.states[t,i,j] = -1 # put it out and clear it

                    # propaga o fogo imediatamente em vegetação rasteira
                    if (grid.states[t-1,i+1,j] == 1) and not(u):
                        grid.states[t,i+1,j] = 100
                    if (grid.states[t-1,i-1,j] == 1) and not(d):
                        grid.states[t,i-1,j] = 100
                    if (grid.states[t-1,i,j+1] == 1) and not(l):
                        grid.states[t,i,j+1] = 100
                    if (grid.states[t-1,i,j-1] == 1) and not(r):
                        grid.states[t,i,j-1] = 100
                    
                    # florestas demoram mais para queimar
                    if (grid.states[t-1,i+1,j] == 10) and not(u):
                        grid.states[t,i+1,j] = grid.states[t-1,i+1,j]/10
                    if (grid.states[t-1,i-1,j] == 10) and not(d):
                        grid.states[t,i-1,j] = grid.states[t-1,i-1,j]/10
                    if (grid.states[t-1,i,j+1] == 10) and not(l):
                        grid.states[t,i,j+1] = grid.states[t-1,i,j+1]/10
                    if (grid.states[t-1,i,j-1] == 10) and not(r):
                        grid.states[t,i,j-1] = grid.states[t-1,i,j-1]/10
                    
                    # não é combustível: aceiro/terra e água
                    # permanece com mesmo valor: não queima
                    if (grid.states[t-1,i+1,j] <= 0) and not(u):
                        grid.states[t,i+1,j] = grid.states[t-1,i+1,j]
                    if (grid.states[t-1,i-1,j] <= 0) and not(d):
                        grid.states[t,i-1,j] = grid.states[t-1,i-1,j]
                    if (grid.states[t-1,i,j+1] <= 0) and not(l):
                        grid.states[t,i,j+1] = grid.states[t-1,i,j+1]
                    if (grid.states[t-1,i,j-1] <= 0) and not(r):
                        grid.states[t,i,j-1] = grid.states[t-1,i,j-1]
                    
                    # área protegida: meta é evitá-la
                    # se chegar nela, a simulação finaliza
                    if (grid.states[t-1,i+1,j] == 2651) and not(u):
                        grid.states[t,i+1,j] = 100
                        found_protected = True
                    if (grid.states[t-1,i-1,j] == 2651) and not(d):
                        grid.states[t,i-1,j] = 100
                        found_protected = True
                    if (grid.states[t-1,i,j+1] == 2651) and not(l):
                        grid.states[t,i,j+1] = 100
                        found_protected = True
                    if (grid.states[t-1,i,j-1] == 2651) and not(r):
                        grid.states[t,i,j-1] = 100
                        found_protected = True
                    
                
                if grid.states[t-1,i,j] == -2: # água -> permanece
                    grid.states[t,i,j] = -2
                    
                if grid.states[t-1,i,j] == 0: # terra/aceiro -> permanece
                    grid.states[t,i,j] = 0

In [ ]:
colored = np.zeros((grid.total_time,*grid.terrain_size,3),dtype=np.uint8)

# Cor
for t in range(grid.states.shape[0]):
    for i in range(grid.states[t].shape[0]):
        for j in range(grid.states[t].shape[1]):
            value = grid.states[t,i,j].copy()

            if value == -2:
                colored[t,i,j] = grid.blue # água
            elif value == -1:
                colored[t,i,j] = grid.black # queimada
            elif value == 0:
                colored[t,i,j] = grid.brown # terra/aceiro
            elif value == 1:
                colored[t,i,j] = grid.green # vegetação rasteira
            elif value == 10:
                colored[t,i,j] = grid.forest # vegetação
            elif value == 100:
                colored[t,i,j] = grid.red # fogo
            elif value == 2651:
                colored[t,i,j] = grid.gold # área protegida

# Crop
cropped = colored[:200, 1:grid.terrain_size[0]-1,1:grid.terrain_size[1]-1]

imageio.mimsave('./video.gif', cropped)